# В этом файле находится код, которым я парсила авито.
### 1. Импортирую необходимые библиотеки

In [ ]:
import json
import logging
import pickle
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

### 2. Собираю ссылки с помощью Selenium и BeautifulSoup

Прохожу по первым 50 страницам, и собираю ссылки на объявления. Из-за проверки на робота получилось собрать не 2500, а немногим меньше ссылок, что меня, однако, устроило.

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

all_links = []

for i in range(50):
    url = f"https://www.avito.ru/all/rasteniya?cd=1&p={i}&q=%D0%BA%D0%BE%D0%BC%D0%BD%D0%B0%D1%82%D0%BD%D1%8B%D0%B5+%D1%86%D0%B2%D0%B5%D1%82%D1%8B"
    logger.info(f"Открытие страницы: {url}")
    driver.get(url)

    time.sleep(5)

    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')

    links = soup.select('div.iva-item-body-KLUuy > div.iva-item-titleStep-pdebR > div > a.styles-module-root-YeOVk.styles-module-root_noVisited-MpiGq')

    logger.info(f"Найдено {len(links)} ссылок")
    for link in links:
        href = link.get('href')
        all_links.append(href)

#### 2.1. Сохраняю ссылки на будущее

In [ ]:
# with open('links_set.pkl', 'wb') as file:
#     pickle.dump(set(all_links), file)

with open('links_set.pkl', 'rb') as file:
    all_links = pickle.load(file)

### 3. Собираю информацию с каждой из объявлений, переходя по ссылкам

Что я собираю:
- Цена
- Несколько ли товаров в продаже
- Средняя оценка продавца
- Число отзывов о продавце
- Местоположение товара
- Время публикации
- Число просмотров у объявления
- Число просмотров у объявления за последний день (обновлено 2 июня 2024)


In [ ]:
plant_data_dict = {}
all_links_list = sorted(list(all_links))

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Запуск в фоновом режиме
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

for i in range(len(all_links_list)):
    url = 'https://www.avito.ru' + all_links_list[i]
    
    if url not in plant_data_dict:
        driver.get(url)

        time.sleep(5)

        page_source = driver.page_source

        # Закрываем драйвер
        # driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')

        price_elements = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-right-rxJqW.style-itemViewContentRightRedesign-KspMX > div > div > div > div.style-item-view-price-block-WSyYk.style-withRedesign-lv9A4 > div > div.styles-module-theme-rOnN1 > div > div:nth-child(1) > div > span > span > span.styles-module-size_xxxl-MrhiK')
        multiple_goods = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-right-rxJqW.style-itemViewContentRightRedesign-KspMX > div > div > div > div.style-item-view-price-block-WSyYk.style-withRedesign-lv9A4 > div > div.styles-module-theme-rOnN1 > div > p > span:nth-child(2)')
        seller_rating = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-right-rxJqW.style-itemViewContentRightRedesign-KspMX > div > div > div > div:nth-child(2) > div.styles-module-theme-rOnN1 > div > div > div > div:nth-child(1) > div.style-seller-info-prop-dafPB.undefined.style-two-col-_3AFC > div.style-seller-info-col-PETb_ > div:nth-child(1) > div.style-seller-info-rating-xHI5T.seller-info-rating > span.style-seller-info-rating-score-C0y96')
        num_reviews = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-right-rxJqW.style-itemViewContentRightRedesign-KspMX > div > div > div > div:nth-child(2) > div.styles-module-theme-rOnN1 > div > div > div > div:nth-child(1) > div.style-seller-info-prop-dafPB.undefined.style-two-col-_3AFC > div.style-seller-info-col-PETb_ > div:nth-child(1) > div.style-seller-info-rating-xHI5T.seller-info-rating > span.styles-module-size_s-AGMw8')
        address = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-left-bb5Ih.style-itemViewContentLeftRedesign-eaVKk > div.style-item-view-main-tKI1S.js-item-view-main.style-item-min-height-TJwyJ > div:nth-child(1) > div.style-item-view-block-SEFaY.style-item-view-map-rppAn.style-new-style-iX7zV > div > div.style-item-map-location-wbfMT > div.style-item-address-KooqC > div > span')
        date_time = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-left-bb5Ih.style-itemViewContentLeftRedesign-eaVKk > div.style-item-view-main-tKI1S.js-item-view-main.style-item-min-height-TJwyJ > div.style-item-socials-container-_aa1d.style-new-style-eTWEE > div > article > p > span:nth-child(2)')
        views = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-left-bb5Ih.style-itemViewContentLeftRedesign-eaVKk > div.style-item-view-main-tKI1S.js-item-view-main.style-item-min-height-TJwyJ > div.style-item-socials-container-_aa1d.style-new-style-eTWEE > div > article > p > span:nth-child(3) > span:nth-child(1)')
        new_views = soup.select('#app > div > div.index-module-root-_rU83.index-module-responsive-rN34p.index-module-page_default-NzRy9.index-module-page_default_wide-_a20J > div:nth-child(1) > div > div.style-item-view-PCYlM > div.style-item-view-content-SDgKX.style-itemViewContentRedesign-NdZwn > div.style-item-view-content-left-bb5Ih.style-itemViewContentLeftRedesign-eaVKk > div.style-item-view-main-tKI1S.js-item-view-main.style-item-min-height-TJwyJ > div.style-item-socials-container-_aa1d.style-new-style-eTWEE > div > article > p > span:nth-child(3) > span:nth-child(2)')
        data_list = [price_elements, multiple_goods, seller_rating, num_reviews, address, date_time, views, new_views]

        for j in range(8):
            if len(data_list[j]) != 0:
                data_list[j] = data_list[j][0].get_text(strip=True)
            else:
                data_list[j] = pd.NA
        
        plant_data_dict[url] = plant_data_dict.get(url, data_list)

#### 3.1. Сохраняю данные

In [ ]:
# Нашла в интернете способ справиться с ошибкой "TypeError: Object of type NAType is not JSON serializable":
# https://stackoverflow.com/questions/65411519/typeerror-object-of-type-natype-is-not-json-serializable

# def convert_NA_to_none(obj):
#     if isinstance(obj, pd.NA.__class__):
#         return None
#     if isinstance(obj, dict):
#         return {k: convert_NA_to_none(v) for k, v in obj.items()}
#     if isinstance(obj, list):
#         return [convert_NA_to_none(i) for i in obj]
#     return obj

# Конвертируем NAType объекты в None
# converted_data = convert_NA_to_none(plant_data_dict)

# Сохраняем наши данные в JSON
# with open('plant_data_dict.json', 'w') as json_file:
#     json.dump(converted_data, json_file)

# Загружаем данные из JSON
with open('plant_data_dict.json', 'r') as json_file:
    plant_data_dict = json.load(json_file)